In [37]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()


In [50]:
BATCH_SIZE = 5
NUM_BOXES = 5
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
IMAGE_DEPTH = 256
CHANNELS = 3
CROP_SIZE = (24, 24)

image = tf.random.normal(shape=(BATCH_SIZE, IMAGE_DEPTH, IMAGE_HEIGHT, IMAGE_WIDTH, CHANNELS))
boxes = tf.constant([
    [0.00608575,  0.2796725, 0.07143939, 0.6068903, 0.28775442,  0.00810707],
    [0.5730202,  0.04832733, 0.30627942, 0.5769557,  0.80076957, 0.47625685],
    [0.5588428, 0.13811588,  0.6499971, 0.87894094,  0.7377615, 0.85128677],
    [0.5102949,  0.24102402,  0.09359622,  0.8523874,  0.9471787, 0.8325567],
    [0.3673197, 0.0059309,  0.723763, 0.62676334,  0.8700967,  0.79733527]
])
box_indices = tf.random.uniform(shape=(NUM_BOXES,), minval=0, maxval=BATCH_SIZE, dtype=tf.int32)

_, image_depth, image_height, image_width, _ = image.shape
images_by_indices = tf.gather(image, box_indices)
boxes_unnormalized = tf.cast(boxes * [
    image_depth - 1,
    image_height - 1,
    image_width - 1,
    image_depth - 1,
    image_height - 1,
    image_width - 1
], tf.int32)

def crop_step(i, result):
    test = images_by_indices[i][boxes_unnormalized[i][0]:boxes_unnormalized[i][1]]
    # print(test)
    # print(result)
    # concat = tf.concat([result, a[i][b[i][0]:b[i][1]]], 0)
    print("Box:", boxes_unnormalized[i])
    # print("Image", images_by_indices[i])
    print("Image shape:", images_by_indices[i].shape)
    print("To concat:", images_by_indices[i][boxes_unnormalized[i][0]:boxes_unnormalized[i][2], boxes_unnormalized[i][1]:boxes_unnormalized[i][3], boxes_unnormalized[i][2]:boxes_unnormalized[i][4], :])
    return [tf.add(i, 1), tf.concat([result, [images_by_indices[i][boxes_unnormalized[i][0]:boxes_unnormalized[i][2], boxes_unnormalized[i][1]:boxes_unnormalized[i][3], boxes_unnormalized[i][2]:boxes_unnormalized[i][4]]]], 0)]

result = tf.ragged.constant([[[[[]]]]], tf.float32)
print(result.shape)
i = tf.constant(0)

[i, result] = tf.while_loop(
    lambda i, _: i < tf.shape(a)[0],
    lambda i, result: crop_step(i, result),
    [i, result]
)

(1, ?, ?, ?, ?)
Box: tf.Tensor([  1  71  18 154  73   2], shape=(6,), dtype=int32)
Image shape: (256, 256, 256, 3)
To concat: tf.Tensor(
[[[[-1.88914482e-02 -3.73598635e-01  1.33556569e+00]
   [ 1.02903807e+00  1.39665592e+00  5.67004561e-01]
   [ 4.92625147e-01  6.32201850e-01  6.10987663e-01]
   ...
   [-3.01368922e-01 -5.11892557e-01  7.63358891e-01]
   [-4.22737777e-01  1.05871463e+00 -4.48081642e-01]
   [-8.06312978e-01  4.46544051e-01  1.09053981e+00]]

  [[ 1.35715222e+00 -4.84016150e-01 -6.46151751e-02]
   [ 9.63807628e-02  1.00851905e+00 -1.39565289e+00]
   [ 1.51181972e+00  2.92987257e-01  6.04629338e-01]
   ...
   [ 9.93247151e-01  2.11695686e-01  1.35880363e+00]
   [-1.01523980e-01  2.28788590e+00  8.45959544e-01]
   [-1.51495135e+00  1.81400239e-01 -6.97771788e-01]]

  [[ 2.00430560e-03 -1.75811529e+00  6.43339872e-01]
   [-2.10071063e+00  8.70611429e-01  1.03773201e+00]
   [-7.44432390e-01  1.06472528e+00 -2.84728599e+00]
   ...
   [-1.54284692e+00 -3.26708674e-01 -1.0975

In [39]:
def crop_and_resize(
    image,
    boxes,
    box_ind=None,
    crop_size=None,
    method='bilinear',
    extrapolation_value=0,
    name=None,
    box_indices=None):

    pass

In [40]:
t = tf.constant([[[1, 1, 1], [2, 2, 2]],
                 [[3, 3, 3], [4, 4, 4]],
                 [[5, 5, 5], [6, 6, 6]]])

tf.slice(t, [1, 0, 0], [1, 1, 3])  # [[[3, 3, 3]]]

<tf.Tensor: id=8373, shape=(1, 1, 3), dtype=int32, numpy=array([[[3, 3, 3]]], dtype=int32)>

In [41]:
a = tf.constant([[1, 2, 3], [4, 5, 6]])
b = tf.constant([0, 0, 1])

tf.gather(a, b)

<tf.Tensor: id=8377, shape=(3, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [1, 2, 3],
       [4, 5, 6]], dtype=int32)>

In [42]:
a = np.array([[1, 2, 3], [4, 5, 6]])
b = np.array([0, 0, 1])

a[b]

array([[1, 2, 3],
       [1, 2, 3],
       [4, 5, 6]])

In [43]:
a = np.array([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15]])
b = np.array([[0, 1], [2, 4], [2, 5]])
# b = np.array([slice(0, 1), slice(3, 4), slice(2, 5)])

print(a[0][b[0][0]:b[0][1]])
print(a[1][b[1][0]:b[1][1]])
print(a[2][b[2][0]:b[2][1]])

[1]
[8 9]
[13 14 15]


In [44]:
a = tf.constant([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [11, 12, 13, 14, 15]])
batch_size = tf.shape(a)[0]
b = tf.constant([[0, 1], [3, 4], [2, 5]])

result = tf.ragged.constant([[]], tf.int32)
i = tf.constant(0)

def crop_step(i, result):
    test = a[i][b[i][0]:b[i][1]]
    print(test)
    print(result)
    # concat = tf.concat([result, a[i][b[i][0]:b[i][1]]], 0)
    print("To concat:", a[i][b[i][0]:b[i][1]])
    return [tf.add(i, 1), tf.concat([result, [a[i][b[i][0]:b[i][1]]]], 0)]

[i, result] = tf.while_loop(
    lambda i, _: i < tf.shape(a)[0],
    lambda i, result: crop_step(i, result),
    [i, result]
)

#drop placeholder
result = result[1:]

g = tf.ragged.constant([[1, 2], [3, 4, 5]])

# b = np.array([slice(0, 1), slice(3, 4), slice(2, 5)])

d = tf.range(b[0][0], b[0][1])
e = tf.range(b[1][0], b[1][1])
f = tf.range(b[2][0], b[2][1])
c = tf.map_fn(lambda x: x[0], b)
# tf.map_fn(lambda x: tf.range(x[0], x[1]), b)

tf.Tensor([1], shape=(1,), dtype=int32)
<tf.RaggedTensor [[]]>
To concat: tf.Tensor([1], shape=(1,), dtype=int32)
tf.Tensor([9], shape=(1,), dtype=int32)
<tf.RaggedTensor [[], [1]]>
To concat: tf.Tensor([9], shape=(1,), dtype=int32)
tf.Tensor([13 14 15], shape=(3,), dtype=int32)
<tf.RaggedTensor [[], [1], [9]]>
To concat: tf.Tensor([13 14 15], shape=(3,), dtype=int32)
